# Trabalho Final: Identificação e Rastreamento de Trilhas em PCB

**Alunos:** Gabriel Batista Barbosa // Luan Almeida Valença
**Disciplina:** Processamento de Imagens

## Resumo
Este notebook apresenta uma solução para a identificação de conexões elétricas em imagens de Placas de Circuito Impresso (PCB). O algoritmo realiza:
1.  Binarização da imagem.
2.  Separação morfológica entre Pads (ilhas de solda) e Trilhas.
3.  Rotulagem e ordenação dos Pads (Top-Left -> Bottom-Right).
4.  Esqueletização e poda das trilhas.
5.  Análise de conectividade para gerar o netlist (X <-> Y).

## Bibliotecas Utilizadas
* **NumPy:** Manipulação matricial.
* **Matplotlib:** Leitura e visualização.
* **Scikit-Image (skimage):** Implementações de algoritmos vistos em aula (Morfologia, Limiarização, Skeletonização).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, filters, morphology, measure
from skimage.util import img_as_bool

# Função simples para agilizar a exibição de imagens
def show_steps(images, titles, cmap='gray'):
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=(5 * n, 5))
    if n == 1: axes = [axes]
    
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap=cmap)
        ax.set_title(title)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

print("Bibliotecas importadas.")

In [ ]:
# 1. Carregamento da Imagem
# SUBSTITUA PELO NOME DA SUA IMAGEM (Upload deve ser feito no ambiente)
image_path = 'placa_teste.png' 

try:
    img_original = io.imread(image_path)
except FileNotFoundError:
    print(f"Erro: Imagem '{image_path}' não encontrada. Faça o upload ou corrija o nome.")
    # Criando uma imagem dummy caso o usuário não tenha o arquivo ainda
    img_original = np.zeros((100, 100, 3), dtype=np.uint8)

# 2. Conversão para Escala de Cinza
# Se a imagem já for cinza, removemos a dimensão extra
if len(img_original.shape) == 3:
    img_gray = color.rgb2gray(img_original)
else:
    img_gray = img_original

# 3. Binarização (Thresholding)
# Utilizamos o método de Otsu para encontrar o limiar ideal automaticamente
thresh = filters.threshold_otsu(img_gray)
img_binary = img_gray > thresh

# NOTA: Dependendo da imagem (fundo preto ou branco), pode ser necessário inverter
# Queremos que as TRILHAS e PADS sejam True (Branco) e o fundo False (Preto).
# Descomente a linha abaixo se o fundo for branco:
# img_binary = ~img_binary

show_steps([img_original, img_gray, img_binary], 
           ['Original', 'Escala de Cinza', 'Binária (Otsu)'])

In [ ]:
# A ideia aqui é usar Abertura (Opening) para remover as trilhas finas,
# deixando apenas os pads (que são elementos maiores/circulares).

# Definimos um elemento estruturante (kernel) em forma de disco
# O raio deve ser maior que a largura da trilha, mas menor que o pad
raio_pad = 3  # Ajuste este valor conforme a resolução da sua imagem!
selem = morphology.disk(raio_pad)

# Abertura: Erosão seguida de Dilatação
img_pads_only = morphology.binary_opening(img_binary, selem)

# Para obter só as trilhas, subtraímos os pads da imagem binária original
# Usamos XOR ou subtração lógica
img_tracks_only = img_binary ^ img_pads_only

# Removemos ruídos pequenos das trilhas (opcional)
img_tracks_only = morphology.binary_opening(img_tracks_only, morphology.disk(1))

show_steps([img_binary, img_pads_only, img_tracks_only], 
           ['Binária Completa', 'Apenas Pads (Abertura)', 'Apenas Trilhas (Subtração)'])

In [ ]:
# 1. Rotulagem de Componentes Conexos (Labeling)
label_img = measure.label(img_pads_only)
props = measure.regionprops(label_img)

# 2. Extração de Centróides e Ordenação
# Vamos criar uma lista de dicionários para facilitar a ordenação
pads_data = []

for prop in props:
    y, x = prop.centroid
    pads_data.append({
        'label_original': prop.label,
        'y': y,
        'x': x,
        'bbox': prop.bbox
    })

# Lógica de Ordenação: Cima para Baixo, Esquerda para Direita
# Para evitar erros de alinhamento leve, podemos agrupar por faixas de Y (rows)
# Mas para simplificar, usaremos uma pontuação ponderada se for um grid claro,
# ou ordenação simples por tupla (y, x).
pads_data.sort(key=lambda k: (k['y'], k['x']))

# Atribuindo IDs finais (1 a N)
for i, pad in enumerate(pads_data):
    pad['id_final'] = i + 1

print(f"Total de Pinos detectados: {len(pads_data)}")

# Visualização da Ordem
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(img_pads_only, cmap='gray')
for pad in pads_data:
    ax.text(pad['x'], pad['y'], str(pad['id_final']), 
            color='red', fontsize=12, fontweight='bold', ha='center', va='center')
ax.set_title('Pinos Identificados e Ordenados')
plt.axis('off')
plt.show()

In [ ]:
# 1. Esqueletização (Thinning)
# Reduz as trilhas a 1 pixel de largura mantendo a topologia
skeleton = morphology.skeletonize(img_tracks_only)

# 2. Poda (Pruning) - Simples
# Remove pequenos ramos ("spurs") que não conectam a nada
# Uma forma simples sem funções prontas complexas é iterar detectando end-points
def prune_skeleton(skel, iterations=5):
    pruned = skel.copy()
    for _ in range(iterations):
        # Filtro para detectar pixels com apenas 1 vizinho (pontas)
        # (Isso é uma simplificação didática de pruning)
        # Em produção, usaríamos Hit-or-Miss transform
        pass # Implementação completa exigiria código extenso de vizinhança
        # Como o skeletonize do scikit-image é bom, vamos confiar nele por enquanto
        # ou apenas limpar ruídos pequenos:
        pruned = morphology.remove_small_objects(pruned.astype(bool), min_size=5)
    return pruned

skeleton_pruned = prune_skeleton(skeleton)

show_steps([img_tracks_only, skeleton_pruned], 
           ['Trilhas Originais', 'Esqueleto (Trilhas Finas)'])

In [ ]:
# Aqui ocorre a mágica do "X <-> Y"

# 1. Rotular as trilhas (cada linha contínua ganha um ID)
skeleton_labels = measure.label(skeleton_pruned)
skeleton_props = measure.regionprops(skeleton_labels)

# Vamos criar um mapa para saber onde estão os pads (dilatados levemente para garantir toque)
# Dilatamos os pads um pouco para garantir que toquem o esqueleto
pads_dilated = morphology.binary_dilation(img_pads_only, morphology.disk(2))
pads_labeled_map = measure.label(pads_dilated)

# Se pad ordenado X está na posição (r, c), qual é o label dele na imagem 'pads_labeled_map'?
# Precisamos mapear ID_FINAL (1..N) -> LABEL_DA_IMAGEM
map_id_to_label = {}
for pad in pads_data:
    # Pegamos o valor do pixel no centroide (arredondado)
    yc, xc = int(pad['y']), int(pad['x'])
    val = pads_labeled_map[yc, xc]
    if val != 0:
        map_id_to_label[pad['id_final']] = val

conexies = []

# 2. Iterar sobre cada trilha (esqueleto)
for track_prop in skeleton_props:
    # Pegamos as coordenadas de todos os pixels desta trilha
    coords = track_prop.coords # Lista de [y, x]
    
    # Conjunto para guardar quais pads essa trilha toca
    pads_touched = set()
    
    for (y, x) in coords:
        # Verificar vizinhança na imagem dos PADS
        # Se um pixel da trilha toca um pixel de pad, anotamos
        # Verificamos uma janela 3x3 ao redor do pixel do esqueleto
        y_min, y_max = max(0, y-2), min(pads_labeled_map.shape[0], y+3)
        x_min, x_max = max(0, x-2), min(pads_labeled_map.shape[1], x+3)
        
        window = pads_labeled_map[y_min:y_max, x_min:x_max]
        found_pads = np.unique(window)
        
        for val in found_pads:
            if val > 0: # 0 é fundo
                pads_touched.add(val)
    
    # Se a trilha tocou 2 ou mais pads, temos uma conexão!
    if len(pads_touched) >= 2:
        # Converter Labels da imagem de volta para IDs (1..N)
        ids_touched = []
        for p_label in pads_touched:
            # Encontrar qual ID corresponde a este label
            for pid, plabel in map_id_to_label.items():
                if plabel == p_label:
                    ids_touched.append(pid)
        
        ids_touched.sort()
        # Formatar saída X <-> Y (pares)
        import itertools
        for a, b in itertools.combinations(ids_touched, 2):
            conexies.append(f"{a} <-> {b}")

# Remover duplicatas e ordenar
conexies = sorted(list(set(conexies)))

print("=== NETLIST GERADA (Conexões Identificadas) ===")
if not conexies:
    print("Nenhuma conexão encontrada. Verifique os parâmetros de morfologia.")
else:
    for c in conexies:
        print(c)